In [18]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.models import Sequential
import keras
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tqdm import tqdm
from math import log2
from keras.layers import Embedding, LSTM, Dense,Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

#readind the data


In [2]:
def read_data(file_name):
  data=[]
  with open(file_name, encoding='utf8') as f:
    for line in f:
      line=line.strip()
      data.append(line)
  size = len(data)
  idx_list = [idx + 1 for idx, val in
            enumerate(data) if val == '</s>']
  res = [data[i: j] for i, j in
        zip([0] + idx_list, idx_list + 
        ([size] if idx_list[-1] != size else []))]
  return res


#Finding the maximum length of the input sequence


In [3]:
def find_max_len(data_set):
  list_length=0
  li=[]
  for i in data_set:
    if list_length < len(i):
      list_length = len(i)
  return list_length


#read source data


In [4]:
source_data=read_data("train-source.txt")

#Find the maximum sequence in source data


In [5]:
max_value_source_data=find_max_len(source_data)

In [6]:
max_value_source_data

233

#read the target data

In [7]:
target_data=read_data("train-target.txt")

#find the maximum sequence in target data


In [8]:
max_value_target_data=find_max_len(target_data)

In [10]:
max_value_target_data

223

#Length of the source data and taget data

In [9]:
len(source_data)

45171

In [11]:
len(target_data)

45171

#total number of unique words in thedataset

In [12]:
def total_words(data):
  all_source_data_words=set()
  for sentence in data:
    for word in sentence:
      if word not in all_source_data_words:
        all_source_data_words.add(word)
  return all_source_data_words


In [21]:
source_data_words=sorted(list(total_words(source_data)))

In [22]:
target_data_words=sorted(list(total_words(target_data)))

#initalise the variables

In [25]:
length_encoder_tokens=len(source_data_words)
length_decoder_tokens=len(target_data_words)

In [ ]:
def encode(grams, raw_data):
    chars = sorted(list(set(raw_data)))
    mapping = dict((c, i) for i, c in enumerate(chars))

    sequences = list()
    for line in tqdm(grams, desc='Encoding'):
        # integer encode line
        encoded_seq = [mapping[char] for char in line]
        # store
        sequences.append(encoded_seq)
    return sequences, mapping

In [ ]:
encode()

#Encoder model definition

In [ ]:
model = Sequential()
model.add(Embedding(vocab, 20, input_length=2, trainable=True))
model.add(GRU(25, recurrent_dropout=0.1, dropout=0.1))
model.add(Dense(vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=5)
model.fit(x_tr, y_tr, epochs=10, verbose=1, validation_data=(x_val, y_val), callbacks=stop_early, batch_size=250)
